In [1]:
import numpy as np
from qiskit import Aer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.operators import Z2Symmetries
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock

### Step 1: Define a molecule
Using the sto3g basis with the PySCF driver as an example, we can describe a molecule.

In [2]:
driver = PySCFDriver(atom='Li .0 .0 .0; H .0 .0 1.6', unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis='sto3g')
molecule = driver.run()

### Step 2: Prepare qubit Hamiltonian
Here, we setup the molecular orbitals to be considered and can reduce the problem size when we map to the qubit Hamiltonian. 

We are also able to define the mapping type for the qubit Hamiltonian, which - in some cases - allows one to further reduce the problem size.

In [3]:
map_type = 'bravyi_kitaev'

In [4]:
# please be aware that the idx here with respective to original idx
freeze_list = [0]
remove_list = [-3, -2] # negative number denotes the reverse order

In [5]:
h1 = molecule.one_body_integrals
h2 = molecule.two_body_integrals
nuclear_repulsion_energy = molecule.nuclear_repulsion_energy

num_particles = molecule.num_alpha + molecule.num_beta
num_spin_orbitals = molecule.num_orbitals * 2
print("HF energy: {}".format(molecule.hf_energy - molecule.nuclear_repulsion_energy))
print("# of electrons: {}".format(num_particles))
print("# of spin orbitals: {}".format(num_spin_orbitals))

HF energy: -8.854072040283647
# of electrons: 4
# of spin orbitals: 12


In [6]:
# prepare full idx of freeze_list and remove_list
# convert all negative idx to positive
remove_list = [x % molecule.num_orbitals for x in remove_list]
freeze_list = [x % molecule.num_orbitals for x in freeze_list]
# update the idx in remove_list of the idx after frozen, since the idx of orbitals are changed after freezing
remove_list = [x - len(freeze_list) for x in remove_list]
remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
freeze_list += [x + molecule.num_orbitals for x in freeze_list]

# prepare fermionic hamiltonian with orbital freezing and eliminating, and then map to qubit hamiltonian
# and if PARITY mapping is selected, reduction qubits
energy_shift = 0.0
qubit_reduction = True if map_type == 'parity' else False

ferOp = FermionicOperator(h1=h1, h2=h2)
if len(freeze_list) > 0:
    ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
    num_spin_orbitals -= len(freeze_list)
    num_particles -= len(freeze_list)
if len(remove_list) > 0:
    ferOp = ferOp.fermion_mode_elimination(remove_list)
    num_spin_orbitals -= len(remove_list)

qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)
qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles) if qubit_reduction else qubitOp
qubitOp.chop(10**-10)

print(qubitOp)
print(qubitOp.print_details())

Representation: paulis, qubits: 6, size: 118
IIIIII	(-0.20765933501970935+0j)
IIIIIZ	(-0.0400165091016335+0j)
IIIIZX	(-0.00031512637666365794+0j)
IIIIIX	(0.00031512637666365794+0j)
IIIYYX	(-0.006707552889287746+0j)
IIIXYY	(0.006707552889287746+0j)
IIIIZZ	(-0.12914560136887615+0j)
IIIYYZ	(0.015008967542962912+0j)
IIIXXI	(0.015008967542962912+0j)
IIIZII	(-0.30546618417220134+0j)
IIZZZI	(-0.04001650910163347+0j)
XYYZZI	(0.00031512637666365447+0j)
XXXIII	(0.00031512637666365447+0j)
YIYZZI	(-0.006707552889287758+0j)
XZXIII	(-0.006707552889287758+0j)
IZIIII	(-0.12914560136887615+0j)
ZXIIII	(-0.015008967542962909+0j)
IXIIII	(0.015008967542962909+0j)
ZZIIII	(-0.30546618417220134+0j)
IIIIZI	(0.05263651528550081+0j)
IIIYYI	(0.004191566320147021+0j)
IIIXXZ	(0.004191566320147021+0j)
IIIZIZ	(0.08247949378222487+0j)
IIIYXY	(0.005456156289317787+0j)
IIIXXX	(0.005456156289317787+0j)
IIIZZX	(-0.0028624550781759677+0j)
IIIZIX	(0.0028624550781759677+0j)
IIZZZZ	(0.12182774215821476+0j)
IIZZIX	(-0.01214489

In [7]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright